In [1]:
import dynet as dy
import numpy as np

print(dy.__version__)

2.0


## Create a new computation graph
it is a singleton, we have one at each stage.
dy.renew_cg() clears the current one and starts anew

In [2]:
## ==== Create a new computation graph
# (it is a singleton, we have one at each stage.
# dy.renew_cg() clears the current one and starts anew)
dy.renew_cg()

## Expressions

In [3]:
## ==== Creating Expressions from user input / constants.
x = dy.scalarInput(123)

v = dy.vecInput(3)
v.set([1,2,3])

# Or directly from a numpy array
z = dy.inputTensor([[1,2],[3,4]]) # Row major

In [4]:
## ==== We can take the value of an expression.
# For complex expressions, this will run forward propagation.
print(z.value())
print(z.npvalue())      # as numpy array
print(v.vec_value())    # as vector, if vector
print(x.scalar_value()) # as scalar, if scalar
print(x.value())        # choose the correct one

[[ 1.  2.]
 [ 3.  4.]]
[[ 1.  2.]
 [ 3.  4.]]
[1.0, 2.0, 3.0]
123.0
123.0


## Parameters

In [14]:
## ==== Parameters
# Parameters are things we tune during training.
# Usually a matrix or a vector.

# First we create a parameter collection and add the parameters to it.
m = dy.ParameterCollection()
pW = m.add_parameters((8,8)) # an 8x8 matrix
pb = m.add_parameters(8)

In [37]:
# then we create an Expression out of the parameter collection's parameters
W = dy.parameter(pW)
b = dy.parameter(pb)

# Specifiying parameter initialization
#p = m.add_parameters((3,5), init=0)                         # Creates 3x5 matrix filled with 0 (or any other float)
p = m.add_parameters((3,5), init=dy.UniformInitializer(5))        # Creates 3x5 matrix initialized with U([-a,a])
p = m.add_parameters((3,5), init=dy.NormalInitializer(0,1))   # Creates 3x5 matrix initialized with N(a, b)
p = m.add_parameters((5,5), init=dy.IdentityInitializer())                # Creates 5x5 identity matrix
p = m.add_parameters((3,5), init=dy.GlorotInitializer())                  # Creates 3x5 matrix with glorot init
np_arr = np.zeros((3,5))
p = m.add_parameters(np_arr.shape, init=dy.NumpyInitializer(np_arr)) # Creates 3x5 matrix from a numpy array (size is inferred)

In [39]:
## ===== Lookup parameters
# Similar to parameters, but are representing a "lookup table"
# that maps numbers to vectors.
# These are used for embedding matrices.
# for example, this will have VOCAB_SIZE rows, each of DIM dimensions.
VOCAB_SIZE = 50
DIM = 100
lp  = m.add_lookup_parameters((VOCAB_SIZE, DIM))

In [52]:
# lookup parameters can be initialized from an existing array, i.e:
# m["lookup"].init_from_array(wv)

e5  = dy.lookup(lp, 5)   # create an Expression from row 5.
e5  = lp[5]           # same
e5c = dy.lookup(lp, 5, update=False)  # as before, but don't update when optimizing.
e5

expression 22/2

In [53]:
#e5.set(9)  # now the e5 expression contains row 9
#e5c.set(9)  # ditto

In [54]:
e5  = dy.lookup_batch(lp, [4, 5])   # create a batched Expression from rows 4 and 5.
e5  = lp.batch([4, 5])           # same
e5

expression 25/2

In [57]:
dy.cg_version()

2

In [62]:
# Or lookup parameters
lp = m.add_lookup_parameters((3,5), init=dy.ConstInitializer(0))                         # Creates 3 vectors of dimension 5 filled with zeros
lp = m.add_lookup_parameters((3,5), init=dy.UniformInitializer(5))        # Creates 3 vectors of dimension 5 initialized with U([-a,a])
lp = m.add_lookup_parameters((3,5), init=dy.NormalInitializer(0,1))   # Creates 3 vectors of dimension 5 initialized with N(a, b)
lp = m.add_lookup_parameters((3,5), init=dy.GlorotInitializer())                  # Creates 3 vectors of dimension 5 with glorot init
np_arr = np.zeros((3,5))
p = m.add_lookup_parameters(np_arr.shape, init=dy.NumpyInitializer(np_arr)) # Creates 3 vectors of dimension 5 from a numpy array (first dimension is the lookup dimension)

## Combine expression into complex expressions

In [9]:
e1 = dy.scalarInput(123)
e2 = dy.scalarInput(2)
print(e1.value())
print(e2.value())
# Math
e = e1 + e2
print(e.value())
e = e1 * e2   # for vectors/matrices: matrix multiplication (like e1.dot(e2) in numpy)
print(e.value())
e = e1 - e2
print(e.value())
e = -e1
print(e.value())

123.0
2.0
125.0
246.0
121.0
-123.0


ValueError: Bad input dimensions in AddVectorToAllColumns: [{1} {1}]

In [6]:
e1 = dy.vecInput(3)
e1.set([1,2,3])
print(e1.value())
e2 = dy.vecInput(3)
e2.set([3,2,1])
print(e2.value())

e = dy.dot_product(e1, e2)
print(e.value())
e = dy.cmult(e1, e2)           # component-wise multiply  (like e1*e2 in numpy)
print(e.value())
e = dy.cdiv(e1, e2)            # component-wise divide
print(e.value())

m1 = dy.inputTensor([[1,2],[3,4],[5,6]])
m2 = dy.colwise_add(m1, e2)     # column-wise addition
print(m2.value())

[1.0, 2.0, 3.0]
[3.0, 2.0, 1.0]
10.0
[3.0, 4.0, 3.0]
[0.3333333432674408, 1.0, 3.0]
[[ 4.  5.]
 [ 5.  6.]
 [ 6.  7.]]
